In [1]:
#declarative-base

In [2]:
from sqlalchemy.orm import declarative_base

In [3]:
base = declarative_base()

In [4]:
base.metadata.tables #<-- Core Table 객체-RDBMS

FacadeDict({})

In [5]:
base.registry #<-- Object 등록

In [6]:
from sqlalchemy.schema import Table, Column, ForeignKey
from sqlalchemy.types import Integer, Text

In [8]:
class User(base):
    __tablename__ = 'USER' #앞으로 우리가 사용할 테이블 이름이 될거임.
    #Obj에서 접근할 테이블 이름이고, DBMS 에서 사용될 테이블 이름이 될거임.
    pk = Column('PK', Integer, primary_key=True)
    #이건 Obj 에서 pk  => 이게 실제 DBMS에서 해당 테이블에서 사용될 컬럼 정보가 될거임.
    name =Column('NAME', Text)

    #값들이 나오게 출력해볼거임
    def __repr__(self):
        return f'PK:{self.pk}, NAME:{self.name}'

In [9]:
user1 = User (name='아무거나')

In [10]:
user1

PK:None, NAME:아무거나

In [12]:
type(user1), type(base.metadata.tables['USER'])

(__main__.User, sqlalchemy.sql.schema.Table)

In [13]:
from sqlalchemy import create_engine

In [15]:
engine = create_engine('sqlite:///:memory:', echo = True) #The echo flag is a shortcut to setting up SQLAlchemy logging

In [16]:
base.metadata.create_all(engine)

2023-09-07 09:24:35,006 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-07 09:24:35,007 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("USER")
2023-09-07 09:24:35,008 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-07 09:24:35,009 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("USER")
2023-09-07 09:24:35,011 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-07 09:24:35,013 INFO sqlalchemy.engine.Engine 
CREATE TABLE "USER" (
	"PK" INTEGER NOT NULL, 
	"NAME" TEXT, 
	PRIMARY KEY ("PK")
)


2023-09-07 09:24:35,015 INFO sqlalchemy.engine.Engine [no key 0.00150s] ()
2023-09-07 09:24:35,016 INFO sqlalchemy.engine.Engine COMMIT


In [17]:
from sqlalchemy.orm import sessionmaker

In [18]:
Session = sessionmaker(engine) #엔진 바인딩해줄거임
sess = Session()
# in-memory에 있는 오브젝트들과 실제 코어를 통해서 RDBMS에 전달하는 창구 역할을 해줌. 커뮤니케이터

In [20]:
sess.add(user1) #세션에게 user1 이라는 객체를 등록할거임. 아직은 아무일 안 일어남. 

In [23]:
sess.commit()
#결과는 커밋한거긴 한데
#세션이 관찰하고 있는 오브젝트들을 반영 해서 DBMS에 커밋한거임. 그니까 lazy 커넥팅?을 통해서 이제서야 인서트 된거임.

2023-09-07 09:30:08,377 INFO sqlalchemy.engine.Engine COMMIT


In [22]:
user1
#DB에는 이미 등록이 되어 있는데 pk 값이 비어있는걸 알기때문에 user1을 치는순간 셀렉트해줘서 알아서 넣어준거임. 

2023-09-07 09:29:34,793 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-07 09:29:34,806 INFO sqlalchemy.engine.Engine SELECT "USER"."PK" AS "USER_PK", "USER"."NAME" AS "USER_NAME" 
FROM "USER" 
WHERE "USER"."PK" = ?
2023-09-07 09:29:34,810 INFO sqlalchemy.engine.Engine [generated in 0.00254s] (1,)


PK:1, NAME:아무거나

In [24]:
#이때 커밋했을때 메세지와 user1 찍었을 때 메세지를 봐야함. 

In [25]:
sess.is_modified(user1)

False

In [26]:
sess.dirty #쌓여있는 작업목록을 볼 수 있음

IdentitySet([])

In [27]:
user1.name = '다른값'

In [28]:
user1 #이렇게만 하면 in-memory에 반영된거지 DB에는 반영된거아님.

PK:1, NAME:다른값

In [29]:
sess.dirty

IdentitySet([PK:1, NAME:다른값])

In [30]:
sess.is_modified(user1)

True

In [31]:
sess.commit()

2023-09-07 09:33:01,675 INFO sqlalchemy.engine.Engine UPDATE "USER" SET "NAME"=? WHERE "USER"."PK" = ?
2023-09-07 09:33:01,677 INFO sqlalchemy.engine.Engine [generated in 0.00181s] ('다른값', 1)
2023-09-07 09:33:01,679 INFO sqlalchemy.engine.Engine COMMIT


In [32]:
user1

2023-09-07 09:33:03,697 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-07 09:33:03,697 INFO sqlalchemy.engine.Engine SELECT "USER"."PK" AS "USER_PK", "USER"."NAME" AS "USER_NAME" 
FROM "USER" 
WHERE "USER"."PK" = ?
2023-09-07 09:33:03,713 INFO sqlalchemy.engine.Engine [cached since 208.9s ago] (1,)


PK:1, NAME:다른값

In [33]:
#당장의 값을 수정하는건 DB에 반영되지 않기 때문에 상관없는데, 언제 세션을 통해서 반영을 하느냐가 포인트임!!!

In [34]:
sess.add_all([User(name='2'), User(name='3')])

In [35]:
sess.commit()

2023-09-07 09:34:27,447 INFO sqlalchemy.engine.Engine INSERT INTO "USER" ("NAME") VALUES (?) RETURNING "PK"
2023-09-07 09:34:27,449 INFO sqlalchemy.engine.Engine [generated in 0.00017s (insertmanyvalues) 1/2 (ordered; batch not supported)] ('2',)
2023-09-07 09:34:27,451 INFO sqlalchemy.engine.Engine INSERT INTO "USER" ("NAME") VALUES (?) RETURNING "PK"
2023-09-07 09:34:27,452 INFO sqlalchemy.engine.Engine [insertmanyvalues 2/2 (ordered; batch not supported)] ('3',)
2023-09-07 09:34:27,456 INFO sqlalchemy.engine.Engine COMMIT


In [36]:
userList = sess.query(User).all()

2023-09-07 09:34:42,215 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-07 09:34:42,217 INFO sqlalchemy.engine.Engine SELECT "USER"."PK" AS "USER_PK", "USER"."NAME" AS "USER_NAME" 
FROM "USER"
2023-09-07 09:34:42,218 INFO sqlalchemy.engine.Engine [generated in 0.00113s] ()


In [37]:
len(userList)

3

In [38]:
user1 is userList[0]

True

In [39]:
userList[1]

PK:2, NAME:2

In [40]:
userList[2]

PK:3, NAME:3

In [41]:
userList[1].name = '다른값2'

In [42]:
sess.dirty

IdentitySet([PK:2, NAME:다른값2])

In [43]:
sess.commit()

2023-09-07 09:35:40,971 INFO sqlalchemy.engine.Engine UPDATE "USER" SET "NAME"=? WHERE "USER"."PK" = ?
2023-09-07 09:35:40,972 INFO sqlalchemy.engine.Engine [cached since 159.3s ago] ('다른값2', 2)
2023-09-07 09:35:40,973 INFO sqlalchemy.engine.Engine COMMIT


In [44]:
userList[1]

2023-09-07 09:35:47,069 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-07 09:35:47,070 INFO sqlalchemy.engine.Engine SELECT "USER"."PK" AS "USER_PK", "USER"."NAME" AS "USER_NAME" 
FROM "USER" 
WHERE "USER"."PK" = ?
2023-09-07 09:35:47,071 INFO sqlalchemy.engine.Engine [cached since 372.3s ago] (2,)


PK:2, NAME:다른값2

In [45]:
#새로운 값으로 계속 덮어씌우고 있다는거다. 

In [46]:
#그래서 중요한건 sess.dirty 했을때 무언가 값이 남아있다면 DB랑 싱크가 안맞다는것이므로 커밋을 해줘야하는거임. 

In [48]:
#값을 검색하려고 할때
[user for user in userList if user.name == '3'][0] is userList[2]

True

In [49]:
sess.query(User).where(User.name == '3').one() is userList[2]

2023-09-07 09:39:17,948 INFO sqlalchemy.engine.Engine SELECT "USER"."PK" AS "USER_PK", "USER"."NAME" AS "USER_NAME" 
FROM "USER" 
WHERE "USER"."NAME" = ?
2023-09-07 09:39:17,950 INFO sqlalchemy.engine.Engine [generated in 0.00158s] ('3',)


True

In [50]:
#OOP 관점에서 코드 치는 사람은 굉장히 편한 과정이다.

In [51]:
class Address(base):
    __tablename__ = 'ADDRESS'

    pk = Column('PK', Integer, primary_key=True)
    name = Column('NAME', Text)
    fk = Column('FK', None, ForeignKey('USER.PK'))

    def __repr__(self):
        return f'PK:{self.pk}, NAME:{self.name}, FK:{self.fk}'
        

In [52]:
#등록하기
base.metadata.create_all(engine)

2023-09-07 09:45:11,742 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-07 09:45:11,742 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("USER")
2023-09-07 09:45:11,742 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-07 09:45:11,754 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("ADDRESS")
2023-09-07 09:45:11,755 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-07 09:45:11,756 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("ADDRESS")
2023-09-07 09:45:11,758 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-07 09:45:11,759 INFO sqlalchemy.engine.Engine 
CREATE TABLE "ADDRESS" (
	"PK" INTEGER NOT NULL, 
	"NAME" TEXT, 
	"FK" INTEGER, 
	PRIMARY KEY ("PK"), 
	FOREIGN KEY("FK") REFERENCES "USER" ("PK")
)


2023-09-07 09:45:11,760 INFO sqlalchemy.engine.Engine [no key 0.00095s] ()
2023-09-07 09:45:11,761 INFO sqlalchemy.engine.Engine COMMIT


In [53]:
sess.add(Address(name='주소1', fk=user1.pk))

In [54]:
sess.dirty

IdentitySet([])

In [55]:
sess.commit()

2023-09-07 09:46:12,177 INFO sqlalchemy.engine.Engine INSERT INTO "ADDRESS" ("NAME", "FK") VALUES (?, ?)
2023-09-07 09:46:12,179 INFO sqlalchemy.engine.Engine [generated in 0.00124s] ('주소1', 1)
2023-09-07 09:46:12,181 INFO sqlalchemy.engine.Engine COMMIT


In [56]:
address1 = sess.query(Address).one()

2023-09-07 09:46:31,042 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-07 09:46:31,044 INFO sqlalchemy.engine.Engine SELECT "ADDRESS"."PK" AS "ADDRESS_PK", "ADDRESS"."NAME" AS "ADDRESS_NAME", "ADDRESS"."FK" AS "ADDRESS_FK" 
FROM "ADDRESS"
2023-09-07 09:46:31,046 INFO sqlalchemy.engine.Engine [generated in 0.00148s] ()


In [57]:
for user in userList:
    if user.pk == address1.fk:
        print(user, address1)

PK:1, NAME:다른값 PK:1, NAME:주소1, FK:1
2023-09-07 09:47:00,454 INFO sqlalchemy.engine.Engine SELECT "USER"."PK" AS "USER_PK", "USER"."NAME" AS "USER_NAME" 
FROM "USER" 
WHERE "USER"."PK" = ?
2023-09-07 09:47:00,455 INFO sqlalchemy.engine.Engine [cached since 1046s ago] (2,)
2023-09-07 09:47:00,458 INFO sqlalchemy.engine.Engine SELECT "USER"."PK" AS "USER_PK", "USER"."NAME" AS "USER_NAME" 
FROM "USER" 
WHERE "USER"."PK" = ?
2023-09-07 09:47:00,458 INFO sqlalchemy.engine.Engine [cached since 1046s ago] (3,)


In [58]:
#다 날리고 

In [59]:
engine.dispose()

In [60]:
base.metadata.clear()

In [61]:
base.registry.dispose()

In [63]:
#이제 아무것도 없이 비어있는 상태이다. 
#이 상태에서, 깨끗한 상태에서 작업할 수 있다. 엔진도 sqlite에 연결되어있고

In [64]:
#CITY, SUPPLIER, PART, SELLS 를 ORM 으로 만들기 

In [65]:
base.metadata.tables

FacadeDict({})

In [66]:
base.registry

In [67]:
class City(base):
    __tablename__ = 'CITY'
    pk = Column('PK', Integer, primary_key=True)
    name =Column('NAME', Text)

    def __repr__(self):
        return f'PK:{self.pk}, NAME:{self.name}'

In [68]:
city1 = City(name='Paris')

In [69]:
city1

PK:None, NAME:Paris

In [70]:
type(city1), type(base.metadata.tables['CITY'])

(__main__.City, sqlalchemy.sql.schema.Table)

In [71]:
from sqlalchemy import create_engine

In [72]:
engine = create_engine('sqlite:///:memory:', echo=True)

In [73]:
base.metadata.create_all(engine)

2023-09-07 09:57:32,557 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-07 09:57:32,558 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("CITY")
2023-09-07 09:57:32,559 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-07 09:57:32,560 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("CITY")
2023-09-07 09:57:32,561 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-07 09:57:32,565 INFO sqlalchemy.engine.Engine 
CREATE TABLE "CITY" (
	"PK" INTEGER NOT NULL, 
	"NAME" TEXT, 
	PRIMARY KEY ("PK")
)


2023-09-07 09:57:32,566 INFO sqlalchemy.engine.Engine [no key 0.00080s] ()
2023-09-07 09:57:32,567 INFO sqlalchemy.engine.Engine COMMIT


In [74]:
#CITY, SUPPLIER, PART, SELLS
class Supplier(base):
    __tablename__ = 'SUPPLIER'
    pk = Column('PK', Integer, primary_key=True)
    name =Column('NAME', Text)

    def __repr__(self):
        return f'PK:{self.pk}, NAME:{self.name}'

class Part(base):
    __tablename__ = 'PART'
    pk = Column('PK', Integer, primary_key=True)
    name =Column('NAME', Text)

    def __repr__(self):
        return f'PK:{self.pk}, NAME:{self.name}'

class Sells(base):
    __tablename__ = 'SELLS'
    pk = Column('PK', Integer, primary_key=True)
    name =Column('NAME', Text)

    def __repr__(self):
        return f'PK:{self.pk}, NAME:{self.name}'

In [75]:
supplier1 = Supplier(name='Jane')

In [76]:
supplier1

PK:None, NAME:Jane

In [77]:
part1 = Part(name='Screw')
part1

PK:None, NAME:Screw

In [78]:
meta.remove(Sells)

NameError: name 'meta' is not defined

In [79]:
base.meta.remove(Sells)

AttributeError: type object 'Base' has no attribute 'meta'

In [80]:
metadata.remove(Sells)

NameError: name 'metadata' is not defined

In [81]:
base.metadata.remove(Sells)

AttributeError: type object 'Sells' has no attribute 'schema'

In [82]:
base.metadata.drop_all(Sells)

AttributeError: type object 'Sells' has no attribute '_run_ddl_visitor'

In [83]:
#선생님과 같이 해보기.

In [144]:
engine.dispose()

In [145]:
base.metadata.clear()

In [147]:
base.registry.dispose()

In [148]:
#CITY, SUPPLIER, PART, SELLS
class City(base):
    __tablename__ = 'CITY'
    pk = Column('PK', Integer, primary_key=True)
    name =Column('NAME', Text)

        
class Supplier(base):
    __tablename__ = 'SUPPLIER'
    pk = Column('PK', Integer, primary_key=True)
    name =Column('NAME', Text)
    fk = Column('FK', Integer, ForeignKey('CITY.PK'))


class Part(base):
    __tablename__ = 'PART'
    pk = Column('PK', Integer, primary_key=True)
    name =Column('NAME', Text)


class Sells(base):
    __tablename__ = 'SELLS'
    pk = Column('PK', Integer, primary_key=True)
    name =Column('NAME', Text)
    fk1 = Column('FK1', Integer, ForeignKey('SUPPLIER.PK'))
    fk2 = Column('FK2', Integer, ForeignKey('PART.PK'))

#ORM 에서는 PK가 꼭 필요하다. 

In [149]:
base.metadata.create_all(engine)

2023-09-07 10:44:18,143 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-07 10:44:18,143 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("CITY")
2023-09-07 10:44:18,156 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-07 10:44:18,159 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("CITY")
2023-09-07 10:44:18,159 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-07 10:44:18,161 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("SUPPLIER")
2023-09-07 10:44:18,161 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-07 10:44:18,164 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("SUPPLIER")
2023-09-07 10:44:18,166 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-07 10:44:18,168 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("PART")
2023-09-07 10:44:18,168 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-07 10:44:18,168 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("PART")
2023-09-07 10:44:18,168 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-0

In [150]:
City, Supplier, Part, Sells

(__main__.City, __main__.Supplier, __main__.Part, __main__.Sells)

In [98]:
# engine = create_engine('sqlite:///:memory:', echo=True)
# Session = sessionmaker(engine)
# sess = Session()

In [151]:
sess.add_all([City(name='도시1'), City(name='도시2'),
            Part(name='부품1'), Part(name='부품2'),
            Supplier(name='지점1'),Supplier(name='지점2')])

In [152]:
sess.commit()

2023-09-07 10:44:24,070 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-07 10:44:24,086 INFO sqlalchemy.engine.Engine INSERT INTO "CITY" ("NAME") VALUES (?) RETURNING "PK"
2023-09-07 10:44:24,089 INFO sqlalchemy.engine.Engine [generated in 0.00017s (insertmanyvalues) 1/2 (ordered; batch not supported)] ('도시1',)
2023-09-07 10:44:24,090 INFO sqlalchemy.engine.Engine INSERT INTO "CITY" ("NAME") VALUES (?) RETURNING "PK"
2023-09-07 10:44:24,092 INFO sqlalchemy.engine.Engine [insertmanyvalues 2/2 (ordered; batch not supported)] ('도시2',)
2023-09-07 10:44:24,097 INFO sqlalchemy.engine.Engine INSERT INTO "PART" ("NAME") VALUES (?) RETURNING "PK"
2023-09-07 10:44:24,097 INFO sqlalchemy.engine.Engine [generated in 0.00016s (insertmanyvalues) 1/2 (ordered; batch not supported)] ('부품1',)
2023-09-07 10:44:24,097 INFO sqlalchemy.engine.Engine INSERT INTO "PART" ("NAME") VALUES (?) RETURNING "PK"
2023-09-07 10:44:24,097 INFO sqlalchemy.engine.Engine [insertmanyvalues 2/2 (ordered; batch not su

In [101]:
sess.rollback()

In [135]:
sess.dirty

IdentitySet([])

In [112]:
#이제 수정을 해볼거임 Relationship 을 이용해서

In [113]:
#일단 인스턴스들이 다 등록되어있을테니 날릴거임

In [137]:
base.registry.dispose()

In [136]:
from sqlalchemy.orm import relationship

In [115]:
#DB에는 남아있고, 메타에도 있는데 단지 클래스만 날아간거임. 

In [138]:
#CITY, SUPPLIER, PART, SELLS
class City(base):
    __tablename__ = 'CITY'
    __table_args__ = {'extend_existing' : True}
    pk = Column('PK', Integer, primary_key=True)
    name =Column('NAME', Text)
    supplier = relationship('Supplier', back_populates='city', uselist=True)

        
class Supplier(base):
    __tablename__ = 'SUPPLIER'
    __table_args__ = {'extend_existing' : True}
    pk = Column('PK', Integer, primary_key=True)
    name =Column('NAME', Text)
    fk = Column('FK', Integer, ForeignKey('CITY.PK'))
    city = relationship('City', back_populates='supplier')


class Part(base):
    __tablename__ = 'PART'
    __table_args__ = {'extend_existing' : True}
    pk = Column('PK', Integer, primary_key=True)
    name =Column('NAME', Text)


class Sells(base):
    __tablename__ = 'SELLS'
    __table_args__ = {'extend_existing' : True}
    pk = Column('PK', Integer, primary_key=True)
    fk1 = Column('FK1', Integer, ForeignKey('SUPPLIER.PK'))
    fk2 = Column('FK2', Integer, ForeignKey('PART.PK'))

In [143]:
city.supplier

NameError: name 'city' is not defined

In [140]:
city

NameError: name 'city' is not defined

In [141]:
City

__main__.City

In [142]:
City.nodes

AttributeError: type object 'City' has no attribute 'nodes'

In [123]:
len(City.nodes)

TypeError: object of type 'InstrumentedAttribute' has no len()

In [124]:
len(city.nodes)

NameError: name 'city' is not defined

In [125]:
sess.dirty

IdentitySet([])

In [126]:
sess.commit()

In [ ]:
sess.query(Supplier).all()

In [153]:
#세션 포함 다 없앨거임...... 하.........

In [206]:
engine.dispose()
sess.close()
base.registry.dispose()
base.metadata.clear()

In [155]:
#노래 예제 해보기

In [207]:
engine = create_engine('sqlite:///:memory:', echo=True)
base = declarative_base()
Session = sessionmaker(engine)
sess = Session()

In [157]:
#Lazy Connecting

In [209]:
class Artist(base):
    __tablename__ = 'ARTIST'
    pk = Column('PK', Integer, primary_key=True)
    name= Column('NAME', Text)
    albums = relationship('Album', back_populates='artist', uselist=True)

    def addAlbum(self, name):
        album = Album(name=name)
        sess.add(album)
        sess.commit()
        self.albums.append(album)

    # def delAlbum(self, name):

    #     album = Album(name=name)
    #     sess.delete(album)
    #     sess.commit()
    #     self.albums.remove(album)

    def delAlbum(self, name):
    # 데이터베이스에서 해당 앨범을 조회
        album_to_delete = sess.query(Album).filter_by(name=name).first()

        if album_to_delete:
            # 데이터베이스에서 삭제
            sess.delete(album_to_delete)
            sess.commit()
            # 클래스의 albums 리스트에서도 제거
            self.albums.remove(album_to_delete)
        else:
            print(f"Album with name '{name}' not found.")
        
    def __repr__(self):
        return f'{self.name}, {len(self.albums)}'

class Album(base):
    __tablename__ = 'ALBUM'
    pk = Column('PK', Integer, primary_key=True)
    name= Column('NAME', Text)
    fk = Column('FK', ForeignKey('ARTIST.PK'))
    artist = relationship('Artist', back_populates='albums')

    def __repr__(self):
        return f'{self.name}, {self.artist.name}'

In [210]:
base.metadata.create_all(engine)

2023-09-07 11:19:49,387 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-07 11:19:49,387 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("ARTIST")
2023-09-07 11:19:49,388 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-07 11:19:49,390 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("ARTIST")
2023-09-07 11:19:49,391 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-07 11:19:49,394 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("ALBUM")
2023-09-07 11:19:49,395 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-07 11:19:49,397 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("ALBUM")
2023-09-07 11:19:49,398 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-07 11:19:49,400 INFO sqlalchemy.engine.Engine 
CREATE TABLE "ARTIST" (
	"PK" INTEGER NOT NULL, 
	"NAME" TEXT, 
	PRIMARY KEY ("PK")
)


2023-09-07 11:19:49,401 INFO sqlalchemy.engine.Engine [no key 0.00089s] ()
2023-09-07 11:19:49,402 INFO sqlalchemy.engine.Engine 
CREATE TABLE "ALBUM" (
	"PK" INTEGER NOT N

In [211]:
artist1 = Artist(name='가수1')
sess.add(artist1)
sess.commit

<bound method Session.commit of <sqlalchemy.orm.session.Session object at 0x00000185B43C3790>>

In [212]:
artist1.addAlbum('앨범1')

2023-09-07 11:19:53,423 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-07 11:19:53,426 INFO sqlalchemy.engine.Engine INSERT INTO "ARTIST" ("NAME") VALUES (?)
2023-09-07 11:19:53,427 INFO sqlalchemy.engine.Engine [generated in 0.00132s] ('가수1',)
2023-09-07 11:19:53,430 INFO sqlalchemy.engine.Engine INSERT INTO "ALBUM" ("NAME", "FK") VALUES (?, ?)
2023-09-07 11:19:53,432 INFO sqlalchemy.engine.Engine [generated in 0.00157s] ('앨범1', None)
2023-09-07 11:19:53,434 INFO sqlalchemy.engine.Engine COMMIT
2023-09-07 11:19:53,437 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-07 11:19:53,437 INFO sqlalchemy.engine.Engine SELECT "ARTIST"."PK" AS "ARTIST_PK", "ARTIST"."NAME" AS "ARTIST_NAME" 
FROM "ARTIST" 
WHERE "ARTIST"."PK" = ?
2023-09-07 11:19:53,437 INFO sqlalchemy.engine.Engine [generated in 0.00143s] (1,)
2023-09-07 11:19:53,437 INFO sqlalchemy.engine.Engine SELECT "ALBUM"."PK" AS "ALBUM_PK", "ALBUM"."NAME" AS "ALBUM_NAME", "ALBUM"."FK" AS "ALBUM_FK" 
FROM "ALBUM" 
WHERE ? = 

In [213]:
artist1.albums[0]

2023-09-07 11:19:55,952 INFO sqlalchemy.engine.Engine SELECT "ALBUM"."PK" AS "ALBUM_PK", "ALBUM"."NAME" AS "ALBUM_NAME" 
FROM "ALBUM" 
WHERE "ALBUM"."PK" = ?
2023-09-07 11:19:55,953 INFO sqlalchemy.engine.Engine [generated in 0.00175s] (1,)
2023-09-07 11:19:55,956 INFO sqlalchemy.engine.Engine UPDATE "ALBUM" SET "FK"=? WHERE "ALBUM"."PK" = ?
2023-09-07 11:19:55,959 INFO sqlalchemy.engine.Engine [generated in 0.00156s] (1, 1)
2023-09-07 11:19:55,961 INFO sqlalchemy.engine.Engine SELECT "ALBUM"."PK" AS "ALBUM_PK", "ALBUM"."NAME" AS "ALBUM_NAME", "ALBUM"."FK" AS "ALBUM_FK" 
FROM "ALBUM" 
WHERE "ALBUM"."PK" = ?
2023-09-07 11:19:55,962 INFO sqlalchemy.engine.Engine [generated in 0.00121s] (1,)


앨범1, 가수1

In [175]:
#앨범을 삭제하는 코드를 만들기 

In [214]:
artist1

가수1, 1

In [188]:
#존재하는지부터 확인 후 지워야하지 않을까?

In [215]:
sess.commit

<bound method Session.commit of <sqlalchemy.orm.session.Session object at 0x00000185B43C3790>>

In [217]:
artist1.delAlbum('1')

2023-09-07 11:20:14,964 INFO sqlalchemy.engine.Engine SELECT "ALBUM"."PK" AS "ALBUM_PK", "ALBUM"."NAME" AS "ALBUM_NAME", "ALBUM"."FK" AS "ALBUM_FK" 
FROM "ALBUM" 
WHERE "ALBUM"."NAME" = ?
 LIMIT ? OFFSET ?
2023-09-07 11:20:14,966 INFO sqlalchemy.engine.Engine [cached since 11.46s ago] ('1', 1, 0)
Album with name '1' not found.


In [218]:
artist1

가수1, 0

In [220]:
sess.query(Artist)

In [221]:
sess.query(Album)

In [205]:
sess.commit()

2023-09-07 11:19:03,184 INFO sqlalchemy.engine.Engine COMMIT


In [ ]:
# #### 선생님 풀이 #####
# class Artist(base):
#     __tablename__ = 'ARTIST'
#     pk = Column('PK', Integer, primary_key=True)
#     name= Column('NAME', Text)
#     albums = relationship('Album', back_populates='artist', uselist=True)

#     def addAlbum(self, name):
#         album = Album(name=name)
#         sess.add(album)
#         sess.commit()
#         self.albums.append(album)


#     def delAlbum(self, name):
#         album = sess.query(Album).filter(Album.name=name).one()
        

        
#     def __repr__(self):
#         return f'{self.name}, {len(self.albums)}'

# class Album(base):
#     __tablename__ = 'ALBUM'
#     pk = Column('PK', Integer, primary_key=True)
#     name= Column('NAME', Text)
#     fk = Column('FK', ForeignKey('ARTIST.PK'))
#     artist = relationship('Artist', back_populates='albums')

#     def __repr__(self):
#         return f'{self.name}, {self.artist.name}'

In [222]:
#다시 다 삭제하고 다시...

In [225]:
sess.commit

<bound method Session.commit of <sqlalchemy.orm.session.Session object at 0x00000185B43C3790>>

In [236]:
engine.dispose()
sess.close()
base.registry.dispose()
base.metadata.clear()

In [237]:
engine = create_engine('sqlite:///:memory:', echo=True)
base = declarative_base()
Session = sessionmaker(engine)
sess = Session()

In [ ]:
#기존에서 가져오기
# base.metadata.reflect(engine)
# base.metadata.tables['테이블이름']
# class 어쩌고(base):
#     __table__ = base.metadata.tables['테이블이름']
#     pk = 
#     relationship
    

In [228]:
#HashTag 해보기...

In [ ]:
# base.registry.dispose()

In [239]:
#Artist 1:N Albums
#Post N:M Hashtag <-- N:M의 관계를 만들기 위해서는 관계를 정의하고 있는 별도의 테이블을 하나 써야한다

class Post(base):
    __tablename__ = 'POST'
    pk = Column('PK', Integer, primary_key=True)
    content = Column('CONTENT', Text)
    
    tags = relationship('Tags', back_populates='post', uselist=True)

    def addTags(self, tags): #이건 무조건 리스트라고 가정할게
        tagList = list()
        #태그에 있는것들 하나씩 불러오기
        for tag in tags:
            result = sess.query(Hashtag).filter(Hashtag.name == tags)
            if result.count() >0:
                tagList.append(result.one())
            else:
                newTag = Hashtag(name=tag)
                sess.add(newTag)
                sess.commit()
                tagList.append(newTag)
        #태그 리스트를 돌려고 한다.
        for tag in tagList:
            sess.add(Tags(fk1=self.pk, fk2=tag.pk))
            sess.commit()
            tag.count += 1

                
    def __repr__(self):
        return f'{self.pk}, {len(self.tags)}'


class Hashtag(base):
    __tablename__ = 'HASHTAG'
    pk = Column('PK', Integer, primary_key=True)
    name = Column('NAME', Text)
    cnt = Column('CNT', Integer, server_default=0)
    posts = relationship('Tags', back_populates='hashtag', uselist=True)

    def __repr__(self):
        return f'{self.pk}, {self.name}, {len(self.posts)}'


class Tags(base):
    __tablename__ = 'TAGS'
    pk = Column('PK', Integer, primary_key=True)
    #이제 Posts 와 Hashtag 와의 관계를 FK로 엮어줄거임
    fk1 = Column('FK1', None, ForeignKey('POST.PK'))
    fk2 = Column('FK2', None, ForeignKey('HASHTAG.PK'))

    post = relationship('Post', back_populates='tags')
    hashtag = relationship('Hashtag', back_populates='posts')

ArgumentError: Argument 'arg' is expected to be one of type '<class 'str'>' or '<class 'sqlalchemy.sql.elements.ClauseElement'>' or '<class 'sqlalchemy.sql.elements.TextClause'>', got '<class 'int'>'

In [ ]:
# Object/Class
# Post            Tags           Hashtag
# tags----       post/hashtag  -----posts
# Posts.tags[0/1/2/3/...]

In [230]:
base.metadata.create_all(engine)

2023-09-07 11:38:54,716 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-07 11:38:54,724 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("POST")
2023-09-07 11:38:54,725 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-07 11:38:54,727 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("POST")
2023-09-07 11:38:54,729 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-07 11:38:54,730 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("HASHTAG")
2023-09-07 11:38:54,733 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-07 11:38:54,734 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("HASHTAG")
2023-09-07 11:38:54,735 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-07 11:38:54,737 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("TAGS")
2023-09-07 11:38:54,739 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-07 11:38:54,741 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("TAGS")
2023-09-07 11:38:54,743 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-07 

In [231]:
#되는지 확인해보기.
post1 = Post(content = '내용1')
sess.add(post1)
sess.commit()

InvalidRequestError: Mapper 'Mapper[Tags(TAGS)]' has no property 'posts'.  If this property was indicated from other mappers or configure events, ensure registry.configure() has been called.

In [232]:
tag1 = Hashtag(name='해시태그1')
sess.add(tag1)
sess.commit()

InvalidRequestError: One or more mappers failed to initialize - can't proceed with initialization of other mappers. Triggering mapper: 'Mapper[Post(POST)]'. Original exception was: Mapper 'Mapper[Tags(TAGS)]' has no property 'posts'.  If this property was indicated from other mappers or configure events, ensure registry.configure() has been called.

In [233]:
sess.commit()